# ДЗ к Урок 6.

**Домашнее задание**:

Для выполнения работ $P_1, P_2, P_3$ сельскохозяйственное предприятие может приобрести тракторы марок А и B стоимостью соответственно $c_1$ и $c_2$ ден. ед.  каждый. С использованием новой техники необходимо выполнить не менее $b_1$ условных единиц работы $P_1$, не менее $b_2$ условных единиц работы $P_2$ и не менее  $b_3$ условных единиц работы $P_3$. За рассматриваемый промежуток времени с использованием трактора марки A можно выполнить $a_{11}$ условных единиц работы $P_1$, $a_{21}$ условных единиц работы $P_2$ или $a_{31}$ условных единиц работы $P_3$; с использованием трактора марки Б – $a_{12}$ условных единиц работы $P_1$, $a_{22}$ условных единиц работы $P_2$ или $a_{23}$ условных единиц работы $P_3$. 

Требуется:

Составить экономико-математическую модель, позволяющую найти такой вариант приобретения тракторов той или другой марки, при котором будут выполнены все необходимые работы, а затраты на новую технику будут минимальны;
Пользуясь методом ветвей и границ, найти оптимальный вариант приобретения тракторов.

$
b_1 = 20, b_2 = 190, b_3 = 88, a_{11} = 4, a_{21} = 19, a_{31}=4, a_{12} = 1, a_{22} = 15, a_{32} = 15, c_1= 3, c_2 = 5$

**Решение:**

Через $x_1$ и $x_2$ обозначим количество приобретаемых тракторов марок A и B соответственно.

Целевая функция, выражающая затраты на новую технику:

$f = c_1x_1 + c_2x_2$

Ограничения по выполнению необходимых работ $P_1, P_2, P_3$:

$a_11x_1 + a_12x_2 >= b1$

$a_21x_1 + a_22x_2 >= b2$

$a_31x_1 + a_32x_2 >= b3$

Количество приобретаемых тракторов не может быть отрицательным числом: $x_1, x_2 >= 0$.


Целевая функция будет иметь следующий вид:

$f = 3x_1 + 5x_2 \rightarrow min$

$\left\{ \begin{matrix}
4x_1 + x_2 >= 20 \\
19x_1 + 15x_2 >= 190 \\
4x_1 + 15x_2 >= 88
\end{matrix} \right.$

$x_1 >=0, x_2 >=0$

$x_1, x_2$ - целые. 
Решим полученную задачу симплексным или графическим методом с отброшенным условием целочисленности. Получаем:

$x_1 =  , x_2 = ,  f_{min} =  $

Ответ получился нецелым, поэтому решим задачу линейного программирования методом ветвей и границ.

Обе переменные имеют нецелые значения. Выбор переменной $x_i$  порождает две подзадачи:


**Подзадача 1:**

**Подзадача 2:**

Найдем оптимальный план _-й подзадачи c дополнительным ограничением. Решение задачи симплекс-методом:

$x_1 =  , x_2 = ,  f_{min} =  $

Получили целочисленный ответ – исходная задача целочисленного программирования решена.

Следует закупать _ штук тракторов марки A и _ штук тракторов марки B. При этом затраты на новую технику равна ___.

